# Проект по SQL. Анализ книг

## Описание проблемы:

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

задача — проанализировать базу данных — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Цель: сформулировать ценностное предложение для нового продукта.

## Данные

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
#pd.io.sql.read_sql(query, con = engine)

In [3]:
query= ('''
select * from books
limit 5
''')
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query= ('''
select * from authors
limit 5
''')
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query= ('''
select * from reviews
limit 5
''')
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
query= ('''
select * from ratings
limit 5
''')
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query= ('''
select * from publishers
limit 5
''')
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Анализ данных

In [8]:
# сколько книг вышло после 1 января 2000 года;
query= ('''
select count(book_id)
from books
where publication_date::date>'2000-01-01'
''')
pd.io.sql.read_sql(query, con = engine)

,count
0,819


In [9]:
#Для каждой книги посчитайте количество обзоров и среднюю оценку;
query= ('''
select book_id, title, round(avg(rating),2), count(distinct review_id)
from books 
left join ratings ra using(book_id)
left join reviews re using(book_id)
group by 1
order by 4 desc,3 desc 
''')
pd.io.sql.read_sql(query, con = engine)

,book_id,title,round,count
0,948,Twilight (Twilight #1),3.66,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.41,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.29,6
3,656,The Book Thief,4.26,6
4,734,The Glass Castle,4.21,6
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,4.00,0
996,387,Leonardo's Notebooks,4.00,0
997,221,Essential Tales and Poems,4.00,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,0


In [10]:
#Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
query= ('''
select publisher, count(book_id) as number_of_books
from publishers
inner join books using(publisher_id)
where num_pages>50
group by 1
having count(book_id)=(select count(book_id)
                            from books
                            group by publisher_id
                            order by 1 desc
                            limit 1)
''')
pd.io.sql.read_sql(query, con = engine)

,publisher,number_of_books
0,Penguin Books,42


In [14]:
query= (''' select author, round(avg(rating),2) as rating
from authors inner join books using(author_id)
inner join ratings ra using(book_id)
where book_id in
        (select book_id
        from books inner join ratings ra using(book_id)
        group by 1
        having count(rating_id)>=50)
group by 1 
order by 2 desc
limit 1
                            ''')
pd.io.sql.read_sql(query, con = engine)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.29


In [12]:
#Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
query= ('''

select round(avg(reviews_number)) average_reviews_number from 
    (select r.username, count(review_id) as reviews_number
        from (select username, count(rating_id)
        from ratings
        group by 1
        having count(rating_id)>50) r
    inner join reviews using(username)
    group by 1) q1



                            ''')
pd.io.sql.read_sql(query, con = engine)

,average_reviews_number
0,24.0


## Вывод:

после 1 января 2000 года вышло 819 книг.

книги, с самым большим количеством обзоров:

- Twilight (Twilight #1) - 7
- Harry Potter and the Prisoner of Azkaban 6
- Harry Potter and the Chamber of Secrets 6 
- The Book Thief	6
- The Glass Castle	6

Эти книги можно считать самимыи популярными.

издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books

автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré

среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

**Ценностное предложение для нового продукта:**

исходя из анализа информации о книгах, издательствах, авторах, а также пользовательские обзоры книг, можно вывести то, что новое приложение для тех, кто любит читать, должно обязательно содержать книги, которые попадают в топ рейтинга по количеству отзывов и по оценкам (учесть авторов с самой выосокой оценкой книг (один из таких авторов - J.K. Rowling/Mary GrandPré). Опираться на изательства, выпускающие наибольшее количество книг (одно из них - Penguin Books). Это приложение может служить как онлайн библиотекой, так и мобильной версией для скачивания книг и чтения в оффлайн режиме. 
С другой стороны, это может быть не только библиотекой, но и игрой, созданной по мотивам одной из книг самого высокооцениваниемого автора (J.K. Rowling/Mary GrandPré), что будет являться не менее интересным предложением для книжных фанатов, которые будут готовы погрузиться в игровую атмосферу , созданную по мотивам книг(и) люимого автора.

